# One Hot Encoding

<font color='red'>For handling Nominal categorical data (No rank in the catergories)</font>

**Multicollinearity :** when your input features have a interdependent mathematical relationship within themselves, then it is called as multicollinearity. It is an issue when dealing with linear algorithms like linear reg, logistic reg, etc.
    
**Dummy variable trap trap:** gives one hot encoded features which have multicollinearity. Hence you need to drop one of the encoded column, defualt is first

**OHE using most frequent variables**: if your feature has many categories but some of them are most frequent and some are not much frequent, you can group the laters as OTHER category. This will not explode the dimensions while using one hot encoding.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r'../datasets/cars.csv')
df.head()

,brand,km_driven,fuel,owner,selling_price
0,Maruti,145500,Diesel,First Owner,450000
1,Skoda,120000,Diesel,Second Owner,370000
2,Honda,140000,Petrol,Third Owner,158000
3,Hyundai,127000,Diesel,First Owner,225000
4,Maruti,120000,Petrol,First Owner,130000


In [8]:
df['fuel'].value_counts()

Diesel    4402
Petrol    3631
CNG         57
LPG         38
Name: fuel, dtype: int64

In [3]:
df['owner'].value_counts()

First Owner             5289
Second Owner            2105
Third Owner              555
Fourth & Above Owner     174
Test Drive Car             5
Name: owner, dtype: int64

### 1. OneHotEncoding using Pandas

In [7]:
pd.get_dummies(df, columns=['fuel','owner']).head()

,brand,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,0,1,0,0,1,0,0,0,0
1,Skoda,120000,370000,0,1,0,0,0,0,1,0,0
2,Honda,140000,158000,0,0,0,1,0,0,0,0,1
3,Hyundai,127000,225000,0,1,0,0,1,0,0,0,0
4,Maruti,120000,130000,0,0,0,1,1,0,0,0,0


### 2. K-1 OneHotEncoding

In [6]:
pd.get_dummies(df, columns=['fuel','owner'],drop_first=True).head()

,brand,km_driven,selling_price,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,1,0,0,0,0,0,0
1,Skoda,120000,370000,1,0,0,0,1,0,0
2,Honda,140000,158000,0,0,1,0,0,0,1
3,Hyundai,127000,225000,1,0,0,0,0,0,0
4,Maruti,120000,130000,0,0,1,0,0,0,0


### 3. OneHotEncoding using Sklearn

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,0:4],df.iloc[:,-1],test_size=0.2,random_state=2)

In [10]:
X_train.head()

,brand,km_driven,fuel,owner
5571,Hyundai,35000,Diesel,First Owner
2038,Jeep,60000,Diesel,First Owner
2957,Hyundai,25000,Petrol,First Owner
7618,Mahindra,130000,Diesel,Second Owner
6684,Hyundai,155000,Diesel,First Owner


In [11]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first',
                    sparse=False,
                    dtype=np.int32
                   )

In [12]:
X_train_new = ohe.fit_transform(X_train[['fuel','owner']])
X_test_new = ohe.transform(X_test[['fuel','owner']])

In [13]:
X_train_new.shape

(6502, 7)

In [14]:
np.hstack((X_train[['brand','km_driven']].values,X_train_new))

array([['Hyundai', 35000, 1, ..., 0, 0, 0],
       ['Jeep', 60000, 1, ..., 0, 0, 0],
       ['Hyundai', 25000, 0, ..., 0, 0, 0],
       ...,
       ['Tata', 15000, 0, ..., 0, 0, 0],
       ['Maruti', 32500, 1, ..., 1, 0, 0],
       ['Isuzu', 121000, 1, ..., 0, 0, 0]], dtype=object)

### 4. OneHotEncoding with Top Categories (keeping other less frequent categories as OTHER)

In [19]:
counts = df['brand'].value_counts()
counts

Maruti           2448
Hyundai          1415
Mahindra          772
Tata              734
Toyota            488
Honda             467
Ford              397
Chevrolet         230
Renault           228
Volkswagen        186
BMW               120
Skoda             105
Nissan             81
Jaguar             71
Volvo              67
Datsun             65
Mercedes-Benz      54
Fiat               47
Audi               40
Lexus              34
Jeep               31
Mitsubishi         14
Force               6
Land                6
Isuzu               5
Kia                 4
Ambassador          4
MG                  3
Daewoo              3
Peugeot             1
Ashok               1
Opel                1
Name: brand, dtype: int64

In [20]:
df['brand'].nunique()

32

In [17]:
threshold = 100
repl = counts[counts <= threshold].index

In [18]:
pd.get_dummies(df['brand'].replace(repl, 'uncommon')).sample(5)

,BMW,Chevrolet,Ford,Honda,Hyundai,Mahindra,Maruti,Renault,Skoda,Tata,Toyota,Volkswagen,uncommon
4024,0,0,0,0,1,0,0,0,0,0,0,0,0
377,0,0,0,0,0,0,1,0,0,0,0,0,0
1384,0,0,0,0,1,0,0,0,0,0,0,0,0
2226,0,0,0,0,0,0,1,0,0,0,0,0,0
2264,0,1,0,0,0,0,0,0,0,0,0,0,0
